[Reference](https://www.tensorflow.org/overview/)

In [2]:
import tensorflow as tf
import numpy
tf.enable_eager_execution()

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:

tf.executing_eagerly()

True

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.12783542, -0.31806642,  1.020189  , -0.5339287 , -0.03025557,
         0.48028785,  1.0055939 , -0.06248395,  0.16305816, -0.43862143]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.08540469, 0.05468006, 0.20846155, 0.04406373, 0.07291614,
        0.1214926 , 0.20544112, 0.07060364, 0.08846648, 0.04846996]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.1079018

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.2928 - acc: 0.9148
Epoch 2/5
60000/60000 [==============================] - 2s 28us/sample - loss: 0.1439 - acc: 0.9566
Epoch 3/5
60000/60000 [==============================] - 2s 27us/sample - loss: 0.1067 - acc: 0.9673
Epoch 4/5
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0879 - acc: 0.9727
Epoch 5/5
60000/60000 [==============================] - 2s 27us/sample - loss: 0.0764 - acc: 0.9758


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0731 - acc: 0.9778


[0.07310267583606765, 0.9778]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: id=68331, shape=(5, 10), dtype=float32, numpy=
array([[1.22678884e-07, 1.55507252e-08, 2.82112887e-05, 6.30964932e-05,
        1.05248405e-11, 6.89016304e-07, 5.02756474e-13, 9.99906301e-01,
        8.24129529e-07, 7.52957760e-07],
       [6.13211526e-09, 5.29623576e-05, 9.99754965e-01, 1.60683892e-04,
        1.61423611e-15, 3.13718265e-05, 8.21026358e-09, 4.57854620e-13,
        2.40747022e-09, 2.36440417e-15],
       [7.40010933e-07, 9.98608649e-01, 3.53277064e-05, 1.06623729e-05,
        3.76330048e-04, 1.28824404e-05, 4.27322229e-05, 7.44878780e-04,
        1.67335165e-04, 5.63391154e-07],
       [9.99945045e-01, 3.21123039e-09, 2.16032477e-05, 3.70784619e-07,
        1.15859855e-08, 1.48232048e-05, 4.45280511e-06, 7.63573371e-06,
        4.27028759e-08, 5.89771980e-06],
       [4.43504359e-05, 2.69273359e-09, 5.78480922e-06, 1.01902643e-07,
        9.92822707e-01, 1.04528772e-06, 6.32761703e-06, 5.77568353e-05,
        4.83457870e-06, 7.05709308e-03]], dtype=float32)>